In [4]:
# Import all required libraries
import pandas as pd
import numpy as np
import pingouin as pg
from pathlib import Path
from scipy import stats
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op
import re
import nibabel as nib

# Additional imports for neuroimaging and atlas processing
from nilearn import datasets, masking, input_data, plotting
from nilearn.connectome import ConnectivityMeasure
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [5]:
# Setup directories and plotting theme
RELI_DIR = Path("dset/derivatives/caps/interrater")
FIGURES_DIR = Path("dset/derivatives/figures")
# will use loop later to run over all subjects
OUT_DIR = Path("dset/derivatives/caps")


In [8]:
# Path to the CAPs mask 
mask_filename = Path("dset/derivatives/caps/spatial_masks/sub-Blossom_zscore-weighted-0_CAP_1_pos.nii.gz")

print(f"Mask loaded: {mask_filename}")

# Load the mask to check its properties
mask_img = nib.load(mask_filename)

# Create masker for extracting time series from ROIs
masker = input_data.NiftiLabelsMasker(
    labels_img=mask_img,
    standardize=True,  #z-scores the time series
    memory='nilearn_cache',
    verbose=1
)

Mask loaded: dset/derivatives/caps/spatial_masks/sub-Blossom_zscore-weighted-0_CAP_1_pos.nii.gz
